In [1]:
import pandas as pd
import datetime
import time

In [2]:
#日期列表
# d_list= ['180602', '180603', '180604', '180605', '180606', '180607', '180608', '180609', '180610',\
#           '180611', '180612', '180613', '180614', '180615', '180616', '180617', '180618', '180619', '180620',\
#           '180621', '180622', '180623', '180624', '180625', '180626', '180627', '180628', '180629', '180630',\
#           '180701', '180702', '180703', '180704', '180705', '180706', '180707', '180708', '180709', '180710', \
#           '180711', '180712', '180713', '180714', '180715']

d_list = #['181018', '181019', '181020',
          ['181022', '181023', '181024', '181025', '181026', '181027', '181028', '181029', '181030', \
          '181031', '181101', '181102', '181103', '181104', '181105', '181106', '181107',\
          '181108', '181109', '181110', '181111']
# d_list = ['180716', '180717', '180718', '180719', '180720', '180721', '180722', '180723', '180724', '180725', \
#           '180726', '180727', '180728', '180729', '180730', '180731', '180801', '180802', '180803', '180804', \
#           '180805', '180806', '180807', '180808', '180809', '180810', '180811', '180812', '180813', '180814',\
#           '180815', '180816', '180817', '180818', '180819', '180820', '180821', '180822', '180826',\
#           '180827', '180828', '180829', '180830', '180831', '180901', '180902', '180903', '180904', '180905',\
#           '180906', '180907', '180908', '180909', '180910', '180911', '180912', '180913', '180914', '180915',\
#           '180916', '180917', '180918', '180919', '180920', '180921', '180922', '180923', '180924', '180925', \
#           '180926', '180927', '180928', '180929', '180930', '181001', '181002', '181003', '181004', '181005', \
#           '181006', '181007', '181008', '181009', '181010', '181011', '181012', '181013', '181014', '181015', \
#           '181016', '181017', '181018', '181019', '181020', '181021']

### 统计1-4点事件汇总统计信息

In [6]:
#统计全量特征

start = time.time()

allday_list =[]
allday_index = []
clms = ['new_1','new_4','new_16','new_32','new_2048','o_n_1_4','o_n_1_8','o_n_1_2048','o_n_16_1','o_n_2_1','o_n_2048_4'\
        ,'o_n_2048_8','o_n_32_1','o_n_4_8','o_n_4_2048','o_n_8_16','o_n_8_32']
clms_today = ['td'+cl for cl in clms]

for day in d_list:
    everyday_list=[]


    #day = '180603'
    if day>= '181105'or day in ('181018', '181019'):
        evet = pd.read_csv(r'E:\jupyter\CTM\data_src\evet\evet_%s_.csv'%day)
    else:
        evet = pd.read_csv(r'E:\jupyter\CTM\data_src\evet\evet_%s.csv'%day)
    evet_old_new = evet[['order_id','evt_time','new_value', 'old_value','server']]

    
    
    evet_old_new['hour'] = evet_old_new['evt_time'].apply(lambda x:x[10:13]).apply(int)
    
    evet_old_new = evet_old_new[evet_old_new.hour<4]
    
    evet_old_new = evet_old_new.dropna()
    evet_old_new = evet_old_new.drop_duplicates(keep='first')
    
    evet_old_new['new_value'] = evet_old_new['new_value'].apply(int)
    evet_old_new['old_value'] = evet_old_new['old_value'].apply(int)
    
    #时间截取比较慢，不用
    #day_begain = datetime.datetime.strptime('20'+day+'000000','%Y%m%d%H%M%S')
    #day_therd =  datetime.datetime.strptime('20'+day+'040000','%Y%m%d%H%M%S')
    #evet_old_new['evt_time'] = evet_old_new['evt_time'].apply(lambda x:datetime.datetime.strptime(x,'%Y-%m-%d %H:%M:%S'))
    #evet_old_new = evet_old_new[evet_old_new['evt_time']<=day_therd]
    
    #通过ajob表过滤出当天实例化的作业
    if day>= '181105'or day in ('181018', '181019'):
        ajob = pd.read_csv(r'E:\jupyter\CTM\data_src\ajob\ajob_%s_0_.txt'%day,low_memory=False)
    else:
        ajob = pd.read_csv(r'E:\jupyter\CTM\data_src\ajob\ajob_%s_0.txt'%day,low_memory=False)
    day_int = int(day)
    day_4_hour = int('20'+day +'040000')
    ajob = ajob[ajob.odate==day_int]
    ajob = ajob[ajob['order_time']<=day_4_hour]
    ajob.drop_duplicates('job_name', keep='first', inplace=True)
    ajob = ajob[['order_id','job_name','server']]

    ajob['server'] = ajob['server'].apply(str)
    evet_old_new['server'] = evet_old_new['server'].apply(str)

    ajob['order_id'] = ajob[['order_id','server']].apply(lambda x:x[0]+x[1],axis=1)
    evet_old_new['order_id'] = evet_old_new[['order_id','server']].apply(lambda x:x[0]+x[1],axis=1)

    evet_old_new_filt = evet_old_new.merge(ajob,how='left',on='order_id')
    evet_old_new_filt = evet_old_new_filt[evet_old_new_filt.job_name.notnull()]



    everyday_list.append(evet_old_new[evet_old_new.new_value==1].order_id.count())
    everyday_list.append(evet_old_new[evet_old_new.new_value==4].order_id.count())
    everyday_list.append(evet_old_new[evet_old_new.new_value==16].order_id.count())
    everyday_list.append(evet_old_new[evet_old_new.new_value==32].order_id.count())
    everyday_list.append(evet_old_new[evet_old_new.new_value==2048].order_id.count())
    everyday_list.append(evet_old_new[(evet_old_new.old_value==1)&(evet_old_new.new_value==4)].order_id.count())
    everyday_list.append(evet_old_new[(evet_old_new.old_value==1)&(evet_old_new.new_value==8)].order_id.count())
    everyday_list.append(evet_old_new[(evet_old_new.old_value==1)&(evet_old_new.new_value==2048)].order_id.count())
    everyday_list.append(evet_old_new[(evet_old_new.old_value==16)&(evet_old_new.new_value==1)].order_id.count())    
    everyday_list.append(evet_old_new[(evet_old_new.old_value==2)&(evet_old_new.new_value==1)].order_id.count())
    everyday_list.append(evet_old_new[(evet_old_new.old_value==2048)&(evet_old_new.new_value==4)].order_id.count())
    everyday_list.append(evet_old_new[(evet_old_new.old_value==2048)&(evet_old_new.new_value==8)].order_id.count())
    everyday_list.append(evet_old_new[(evet_old_new.old_value==32)&(evet_old_new.new_value==1)].order_id.count()) 
    everyday_list.append(evet_old_new[(evet_old_new.old_value==4)&(evet_old_new.new_value==8)].order_id.count())
    everyday_list.append(evet_old_new[(evet_old_new.old_value==4)&(evet_old_new.new_value==2048)].order_id.count())
    everyday_list.append(evet_old_new[(evet_old_new.old_value==8)&(evet_old_new.new_value==16)].order_id.count())
    everyday_list.append(evet_old_new[(evet_old_new.old_value==8)&(evet_old_new.new_value==32)].order_id.count()) 
    

    everyday_list.append(evet_old_new_filt[evet_old_new_filt.new_value==1].order_id.count())
    everyday_list.append(evet_old_new_filt[evet_old_new_filt.new_value==4].order_id.count())
    everyday_list.append(evet_old_new_filt[evet_old_new_filt.new_value==16].order_id.count())
    everyday_list.append(evet_old_new_filt[evet_old_new_filt.new_value==32].order_id.count())
    everyday_list.append(evet_old_new_filt[evet_old_new_filt.new_value==2048].order_id.count()) 
    everyday_list.append(evet_old_new_filt[(evet_old_new_filt.old_value==1)&(evet_old_new_filt.new_value==4)].order_id.count())
    everyday_list.append(evet_old_new_filt[(evet_old_new_filt.old_value==1)&(evet_old_new_filt.new_value==8)].order_id.count())
    everyday_list.append(evet_old_new_filt[(evet_old_new_filt.old_value==1)&(evet_old_new_filt.new_value==2048)].order_id.count())
    everyday_list.append(evet_old_new_filt[(evet_old_new_filt.old_value==16)&(evet_old_new_filt.new_value==1)].order_id.count())    
    everyday_list.append(evet_old_new_filt[(evet_old_new_filt.old_value==2)&(evet_old_new_filt.new_value==1)].order_id.count())
    everyday_list.append(evet_old_new_filt[(evet_old_new_filt.old_value==2048)&(evet_old_new_filt.new_value==4)].order_id.count())
    everyday_list.append(evet_old_new_filt[(evet_old_new_filt.old_value==2048)&(evet_old_new_filt.new_value==8)].order_id.count())
    everyday_list.append(evet_old_new_filt[(evet_old_new_filt.old_value==32)&(evet_old_new_filt.new_value==1)].order_id.count()) 
    everyday_list.append(evet_old_new_filt[(evet_old_new_filt.old_value==4)&(evet_old_new_filt.new_value==8)].order_id.count())
    everyday_list.append(evet_old_new_filt[(evet_old_new_filt.old_value==4)&(evet_old_new_filt.new_value==2048)].order_id.count())
    everyday_list.append(evet_old_new_filt[(evet_old_new_filt.old_value==8)&(evet_old_new_filt.new_value==16)].order_id.count())
    everyday_list.append(evet_old_new_filt[(evet_old_new_filt.old_value==8)&(evet_old_new_filt.new_value==32)].order_id.count()) 
    
    allday_list.append(everyday_list)
    
    allday_index.append(day)
    
    print(day,end='  ')
    end = time.time()
    print('    %0.2f minutes'%((end-start)/60))

day_save_df = pd.DataFrame(allday_list,columns = clms+clms_today)

for cl in clms:
    day_save_df['sub'+cl] = day_save_df[[cl,'td'+cl]].apply(lambda x:x[0]-x[1],axis=1)

day_save_df.index = allday_index

day_save_df.to_csv(r'E:\jupyter\CTM\data_processed\evet\evet_1_4hour_count_3.csv',index=True)

181018      0.77 minutes
181019      1.51 minutes
181020      2.21 minutes
181022      2.92 minutes
181023      3.61 minutes
181024      4.31 minutes
181025      5.15 minutes
181026      5.86 minutes
181027      6.57 minutes
181028      7.24 minutes
181029      7.93 minutes
181030      8.67 minutes
181031      9.37 minutes
181101      10.13 minutes
181102      11.01 minutes
181103      11.68 minutes
181104      12.51 minutes
181105      13.27 minutes
181106      13.99 minutes
181107      14.78 minutes
181108      15.56 minutes
181109      16.31 minutes
181110      17.06 minutes
181111      17.77 minutes


### 统计1-8点事件汇总统计信息

In [4]:
#统计全量特征

start = time.time()

allday_list =[]
allday_index = []
clms = ['new_1','new_4','new_16','new_32','new_2048','o_n_1_4','o_n_1_8','o_n_1_2048','o_n_16_1','o_n_2_1','o_n_2048_4'\
        ,'o_n_2048_8','o_n_32_1','o_n_4_8','o_n_4_2048','o_n_8_16','o_n_8_32']
clms_today = ['td'+cl for cl in clms]

for day in d_list:
    everyday_list=[]


    #day = '180603'
    evet = pd.read_csv(r'E:\jupyter\CTM\data_src\evet\evet_%s.csv'%day)   
    evet_old_new = evet[['order_id','evt_time','new_value', 'old_value','server']]

    
    
    evet_old_new['hour'] = evet_old_new['evt_time'].apply(lambda x:x[10:13]).apply(int)
    
    evet_old_new = evet_old_new[evet_old_new.hour<8]
    
    evet_old_new = evet_old_new.dropna()
    evet_old_new = evet_old_new.drop_duplicates(keep='first')
    
    evet_old_new['new_value'] = evet_old_new['new_value'].apply(int)
    evet_old_new['old_value'] = evet_old_new['old_value'].apply(int)
    
    #时间截取比较慢，不用
    #day_begain = datetime.datetime.strptime('20'+day+'000000','%Y%m%d%H%M%S')
    #day_therd =  datetime.datetime.strptime('20'+day+'040000','%Y%m%d%H%M%S')
    #evet_old_new['evt_time'] = evet_old_new['evt_time'].apply(lambda x:datetime.datetime.strptime(x,'%Y-%m-%d %H:%M:%S'))
    #evet_old_new = evet_old_new[evet_old_new['evt_time']<=day_therd]
    
    #通过ajob表过滤出当天实例化的作业
    ajob = pd.read_csv(r'E:\jupyter\CTM\data_src\ajob\ajob_%s_0.txt'%day,low_memory=False)
    day_int = int(day)
    day_4_hour = int('20'+day +'040000')
    ajob = ajob[ajob.odate == day_int]
    ajob = ajob[ajob['order_time']<=day_4_hour]
    ajob.drop_duplicates('job_name', keep='first', inplace=True)
    ajob = ajob[['order_id','job_name','server']]
    

    ajob['server'] = ajob['server'].apply(str)
    evet_old_new['server'] = evet_old_new['server'].apply(str)

    ajob['order_id'] = ajob[['order_id','server']].apply(lambda x:x[0]+x[1],axis=1)
    evet_old_new['order_id'] = evet_old_new[['order_id','server']].apply(lambda x:x[0]+x[1],axis=1)

    evet_old_new_filt = evet_old_new.merge(ajob,how='left',on='order_id')
    evet_old_new_filt = evet_old_new_filt[evet_old_new_filt.job_name.notnull()]



    everyday_list.append(evet_old_new[evet_old_new.new_value==1].order_id.count())
    everyday_list.append(evet_old_new[evet_old_new.new_value==4].order_id.count())
    everyday_list.append(evet_old_new[evet_old_new.new_value==16].order_id.count())
    everyday_list.append(evet_old_new[evet_old_new.new_value==32].order_id.count())
    everyday_list.append(evet_old_new[evet_old_new.new_value==2048].order_id.count())
    everyday_list.append(evet_old_new[(evet_old_new.old_value==1)&(evet_old_new.new_value==4)].order_id.count())
    everyday_list.append(evet_old_new[(evet_old_new.old_value==1)&(evet_old_new.new_value==8)].order_id.count())
    everyday_list.append(evet_old_new[(evet_old_new.old_value==1)&(evet_old_new.new_value==2048)].order_id.count())
    everyday_list.append(evet_old_new[(evet_old_new.old_value==16)&(evet_old_new.new_value==1)].order_id.count())    
    everyday_list.append(evet_old_new[(evet_old_new.old_value==2)&(evet_old_new.new_value==1)].order_id.count())
    everyday_list.append(evet_old_new[(evet_old_new.old_value==2048)&(evet_old_new.new_value==4)].order_id.count())
    everyday_list.append(evet_old_new[(evet_old_new.old_value==2048)&(evet_old_new.new_value==8)].order_id.count())
    everyday_list.append(evet_old_new[(evet_old_new.old_value==32)&(evet_old_new.new_value==1)].order_id.count()) 
    everyday_list.append(evet_old_new[(evet_old_new.old_value==4)&(evet_old_new.new_value==8)].order_id.count())
    everyday_list.append(evet_old_new[(evet_old_new.old_value==4)&(evet_old_new.new_value==2048)].order_id.count())
    everyday_list.append(evet_old_new[(evet_old_new.old_value==8)&(evet_old_new.new_value==16)].order_id.count())
    everyday_list.append(evet_old_new[(evet_old_new.old_value==8)&(evet_old_new.new_value==32)].order_id.count()) 
    

    everyday_list.append(evet_old_new_filt[evet_old_new_filt.new_value==1].order_id.count())
    everyday_list.append(evet_old_new_filt[evet_old_new_filt.new_value==4].order_id.count())
    everyday_list.append(evet_old_new_filt[evet_old_new_filt.new_value==16].order_id.count())
    everyday_list.append(evet_old_new_filt[evet_old_new_filt.new_value==32].order_id.count())
    everyday_list.append(evet_old_new_filt[evet_old_new_filt.new_value==2048].order_id.count()) 
    everyday_list.append(evet_old_new_filt[(evet_old_new_filt.old_value==1)&(evet_old_new_filt.new_value==4)].order_id.count())
    everyday_list.append(evet_old_new_filt[(evet_old_new_filt.old_value==1)&(evet_old_new_filt.new_value==8)].order_id.count())
    everyday_list.append(evet_old_new_filt[(evet_old_new_filt.old_value==1)&(evet_old_new_filt.new_value==2048)].order_id.count())
    everyday_list.append(evet_old_new_filt[(evet_old_new_filt.old_value==16)&(evet_old_new_filt.new_value==1)].order_id.count())    
    everyday_list.append(evet_old_new_filt[(evet_old_new_filt.old_value==2)&(evet_old_new_filt.new_value==1)].order_id.count())
    everyday_list.append(evet_old_new_filt[(evet_old_new_filt.old_value==2048)&(evet_old_new_filt.new_value==4)].order_id.count())
    everyday_list.append(evet_old_new_filt[(evet_old_new_filt.old_value==2048)&(evet_old_new_filt.new_value==8)].order_id.count())
    everyday_list.append(evet_old_new_filt[(evet_old_new_filt.old_value==32)&(evet_old_new_filt.new_value==1)].order_id.count()) 
    everyday_list.append(evet_old_new_filt[(evet_old_new_filt.old_value==4)&(evet_old_new_filt.new_value==8)].order_id.count())
    everyday_list.append(evet_old_new_filt[(evet_old_new_filt.old_value==4)&(evet_old_new_filt.new_value==2048)].order_id.count())
    everyday_list.append(evet_old_new_filt[(evet_old_new_filt.old_value==8)&(evet_old_new_filt.new_value==16)].order_id.count())
    everyday_list.append(evet_old_new_filt[(evet_old_new_filt.old_value==8)&(evet_old_new_filt.new_value==32)].order_id.count()) 
    
    allday_list.append(everyday_list)
    
    allday_index.append(day)
    
    print(day,end='  ')
    end = time.time()
    print('    %0.2f minutes'%((end-start)/60))

day_save_df = pd.DataFrame(allday_list,columns = clms+clms_today)

for cl in clms:
    day_save_df['sub'+cl] = day_save_df[[cl,'td'+cl]].apply(lambda x:x[0]-x[1],axis=1)

day_save_df.index = allday_index

day_save_df.to_csv(r'E:\jupyter\CTM\data_processed\evet\evet_1_8hour_count.csv',index=True)

180602      1.16 minutes
180603      2.62 minutes
180604      4.03 minutes
180605      5.68 minutes
180606      7.37 minutes
180607      8.61 minutes
180608      9.88 minutes
180609      11.06 minutes
180610      12.36 minutes
180611      13.53 minutes
180612      14.79 minutes
180613      16.03 minutes
180614      17.14 minutes
180615      18.59 minutes
180616      19.56 minutes
180617      20.50 minutes
180618      21.50 minutes
180619      22.42 minutes
180620      23.39 minutes
180621      24.44 minutes
180622      25.39 minutes
180623      26.54 minutes
180624      27.43 minutes
180625      28.44 minutes
180626      29.66 minutes
180627      30.88 minutes
180628      31.85 minutes
180629      32.96 minutes
180630      34.25 minutes
180701      35.40 minutes
180702      36.77 minutes
180703      37.96 minutes
180704      39.15 minutes
180705      40.51 minutes
180706      41.70 minutes
180707      42.97 minutes
180708      44.31 minutes
180709      45.64 minutes
180710      46.87 m